In [1]:
# !pip install tensorflow

In [2]:
# !pip install tensorflow_addons

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

from tqdm.auto import tqdm

np.random.seed(12345)
tf.random.set_seed(12345)

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# ========== Data ==========

In [4]:
data = pd.read_csv('dataset2.csv')

In [5]:
data

,x1,x2,y
0,2,1,B
1,2,-1,B
2,-2,2,A
3,1,2,B
4,-2,3,B
5,2,0,B
6,-1,-1,A
7,-2,1,A
8,0,0,A
9,1,-1,A


In [6]:
X = data[['x1', 'x2']]

In [7]:
y = data['y']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

In [9]:
le = LabelEncoder()
y_le_train = le.fit_transform(y_train)
y_le_val = le.transform(y_train)

# ========== Model ==========

In [10]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [11]:
devices_type = [device.device_type for device in tf.config.list_physical_devices()]

devices_type

['CPU']

In [12]:
if 'GPU' in devices_type:
    device_name = '/device:GPU:0'
else:
    device_name = '/device:CPU:0'

In [13]:
best_loss = np.inf

alphas = np.logspace(-3, 2, 100)

for alpha in tqdm(alphas):
    model = Sequential()

    model.add(Dense(5, input_dim=X.shape[1], activation='relu', 
                    kernel_regularizer=regularizers.l1_l2(l1=alpha, l2=0), 
                    bias_regularizer=regularizers.l1_l2(l1=alpha, l2=0)))
    model.add(Dense(4, activation='relu', 
                    kernel_regularizer=regularizers.l1_l2(l1=alpha, l2=0), 
                    bias_regularizer=regularizers.l1_l2(l1=alpha, l2=0)))
    model.add(Dense(1, activation='sigmoid', 
                    kernel_regularizer=regularizers.l1_l2(l1=alpha, l2=0), 
                    bias_regularizer=regularizers.l1_l2(l1=alpha, l2=0)))

    model.compile(loss=BinaryCrossentropy(), optimizer=SGD(learning_rate=1))

    tqdm_callback = tfa.callbacks.TQDMProgressBar(show_epoch_progress=False)

    with tf.device(device_name):
        history = model.fit(X_train, y_le_train, epochs=500, callbacks=[tqdm_callback], verbose=0)  

    y_pred_le_val = model.predict(X_val)
    if np.isnan(y_pred_le_val.min()):
        continue
    loss = log_loss(y_val, y_pred_le_val, labels=np.unique(y))
    
    if loss < best_loss:
        best_loss = loss
        best_model = model
        best_alpha = alpha

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2283: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2283: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

...

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2283: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

In [14]:
best_alpha

0.0025353644939701114

In [15]:
best_model.get_weights()

[array([[ 7.5228256e-04,  1.1085782e+00,  1.5916757e-03, -1.0276217e+00,
          5.8468294e-01],
        [ 1.1133447e-03,  1.1893085e+00,  1.5511820e-03, -1.0131123e+00,
          2.1920633e-04]], dtype=float32),
 array([-3.8997689e-04,  9.4604671e-02,  2.4880583e-03,  1.0752226e+00,
         2.2612093e-03], dtype=float32),
 array([[-2.0163697e-03, -2.0959484e-04, -1.7415776e-03, -2.3653074e-03],
        [ 1.2433537e+00, -1.5670653e-01,  2.3083948e-04,  9.9674755e-01],
        [-4.2430940e-04,  9.3458220e-04,  1.9324236e-03, -1.0272432e-03],
        [-1.1030671e-01,  2.0497026e+00,  1.8965516e-03, -1.4952353e-01],
        [ 1.9715439e-01, -8.1126322e-04,  1.4675129e-04,  2.7873281e-01]],
       dtype=float32),
 array([ 0.0009533 ,  0.08228628, -0.00077255, -0.0015078 ], dtype=float32),
 array([[ 1.4751341e+00],
        [-2.3033814e+00],
        [ 1.4099781e-04],
        [ 1.3341802e+00]], dtype=float32),
 array([0.00037755], dtype=float32)]

# ========== Prediction ==========

In [16]:
X = pd.read_csv('incoming_data2.csv')

In [17]:
X

,x1,x2
0,0,3
1,-2,-1
2,0,1
3,0,-1


In [18]:
best_model.predict(X)

array([[9.9999070e-01],
       [2.6412483e-09],
       [9.8239493e-01],
       [4.3241751e-05]], dtype=float32)

In [19]:
y_pred_le = np.round(best_model.predict(X)).astype(int)
y_pred = le.inverse_transform(y_pred_le.reshape(-1))

y_pred

array(['B', 'A', 'B', 'A'], dtype=object)